In [1]:
path='C:/Users/student/Downloads/데이터들/word2vec자료/'

In [150]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

In [151]:
data=pd.read_csv(path+'Reviews.csv',nrows=100000)
print(len(data))

100000


In [152]:
# data.head(5)
data.info()
data=data[['Text','Summary']]
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
Id                        100000 non-null int64
ProductId                 100000 non-null object
UserId                    100000 non-null object
ProfileName               99996 non-null object
HelpfulnessNumerator      100000 non-null int64
HelpfulnessDenominator    100000 non-null int64
Score                     100000 non-null int64
Time                      100000 non-null int64
Summary                   99998 non-null object
Text                      100000 non-null object
dtypes: int64(5), object(5)
memory usage: 7.6+ MB


,Text,Summary
0,I have bought several of the Vitality canned d...,Good Quality Dog Food
1,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised
2,This is a confection that has been around a fe...,"""Delight"" says it all"
3,If you are looking for the secret ingredient i...,Cough Medicine
4,Great taffy at a great price. There was a wid...,Great taffy


In [153]:
data['Text'] # text 열에 대해 중복 제외한 데이터 개수 출력
data['Text'].nunique()

88426

In [154]:
data.drop_duplicates(subset=['Text'], inplace=True)

In [155]:
data.dropna(axis=0, inplace=True)

In [156]:
# 전처리
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [157]:
stopWords=set(stopwords.words("english"))
len(stopWords)

179

In [158]:
def preprocessSentence(sent, rs=True):
    sent=sent.lower()
    sent=BeautifulSoup(sent, 'lxml').text
    sent=re.sub("\([^)]*\)","",sent)
    sent=re.sub('"',"",sent)
    sent=" ".join([contractions[t] if t in contractions else t for t in sent.split(" ")])
    sent=re.sub("'s\b","",sent) # 소유격 제거 \b는 왼쪽으로 커서를 이동시켜라
    sent=re.sub('[^a-zA-Z]', ' ',sent)
    sent=re.sub("[m]{2,}","mm",sent) # 
    
    if rs:
        tokens=" ".join(word for word in sent.split() if not word in stopWords if len(word)>1)
    
    else: # 불용어 제거 안함(요약)
        tokens=" ".join(word for word in sent.split() if len(word)>1)
    
    return tokens

In [159]:
preprocessSentence('<a>I a""mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm a student</a>')

'amm student'

In [160]:
cleanText=[]
for sent in data['Text']:
    cleanText.append(preprocessSentence(sent))
print(cleanText[:10])

['bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better', 'product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo', 'confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story lewis lion witch wardrobe treat seduces edmund selling brother sisters witch', 'looking secret ingredient robitussin believe found got addition root beer extract ordered made cherry soda flavor medicinal', 'great taffy great price wide assortment yummy taffy delivery quick taffy lover deal', 'got wild hair taffy ordered five pound bag taffy enjoyable many flavors watermelon root beer melon peppermint grape etc complaint bit much red black licorice flavored pieces kids husband lasted

In [161]:
data['Text']=cleanText

In [162]:
cleanSummary=[]
for sent in data['Summary']:
    cleanSummary.append(preprocessSentence(sent))
print(cleanSummary[:10])

['good quality dog food', 'advertised', 'delight says', 'cough medicine', 'great taffy', 'nice taffy', 'great good expensive brands', 'wonderful tasty taffy', 'yay barley', 'healthy dog food']


In [163]:
data['Summary']=cleanSummary

In [164]:
data

,Text,Summary
0,bought several vitality canned dog food produc...,good quality dog food
1,product arrived labeled jumbo salted peanuts p...,advertised
2,confection around centuries light pillowy citr...,delight says
3,looking secret ingredient robitussin believe f...,cough medicine
4,great taffy great price wide assortment yummy ...,great taffy
...,...,...
99995,love buy another box done last one,yummy
99996,late father law used rating system meals parti...,tastes like
99997,favorite brand korean ramen spicy used eating ...,great ramen
99998,like noodles although say spicy somewhat under...,spicy


In [165]:
data.replace("", np.nan, inplace=True)

In [166]:
data.isnull().sum()
data.dropna(axis=0, inplace=True)

In [167]:
len(data)

88134

In [168]:
textLen=[len(s.split()) for s in data['Text']]
summaryLen=[len(s.split()) for s in data['Summary']]

In [169]:
np.min(textLen) # 2
np.max(textLen) # 1235
np.mean(textLen) # 38
np.min(summaryLen) # 1
np.max(summaryLen) # 16
np.mean(summaryLen) # 2,8

2.872115188236095

In [170]:
textMaxLen=50
summaryMaxLen=8

In [171]:
def threshLen(mlen, nlist):
    c=0
    for s in nlist:
        if(len(s.split()) <=mlen):
            c+=1
    print(c/len(nlist))

In [172]:
print(threshLen(textMaxLen, data['Text']))
print(threshLen(summaryMaxLen, data['Summary']))

0.7746726575441941
None
0.9943948986770146
None


In [173]:
data=data[data['Text'].apply(lambda x:len(x.split())<=textMaxLen)]
data=data[data['Summary'].apply(lambda x:len(x.split())<=summaryMaxLen)]

In [174]:
print(len(data)) # 68000

68061


In [176]:
# seq2seq
data['Summary']=data['Summary'].apply(lambda x:'sostoken '+x+' eostoken')
data['Summary']

0        sostoken good quality dog food eostoken
1                   sostoken advertised eostoken
2                 sostoken delight says eostoken
3               sostoken cough medicine eostoken
4                  sostoken great taffy eostoken
                          ...                   
99993              sostoken great stuff eostoken
99994               sostoken good stuff eostoken
99995                    sostoken yummy eostoken
99997              sostoken great ramen eostoken
99998                    sostoken spicy eostoken
Name: Summary, Length: 68061, dtype: object

In [179]:
textData=list(data['Text'])
summaryData=list(data['Summary'])

In [180]:
from sklearn.model_selection import train_test_split

In [181]:
xTrain, xTest, yTrain, yTest=train_test_split(textData, summaryData, test_size=0.2, random_state=42, shuffle=True)

In [183]:
len(xTrain)
len(xTest)

13613

In [184]:
srcToken=Tokenizer()
srcToken.fit_on_texts(xTrain)
# 단어 집합 생성

In [185]:
srcToken.word_index

{'like': 1,
 'good': 2,
 'great': 3,
 'taste': 4,
 'product': 5,
 'love': 6,
 'one': 7,
 'flavor': 8,
 'coffee': 9,
 'would': 10,
 'tea': 11,
 'really': 12,
 'get': 13,
 'amazon': 14,
 'price': 15,
 'best': 16,
 'buy': 17,
 'food': 18,
 'much': 19,
 'little': 20,
 'use': 21,
 'time': 22,
 'find': 23,
 'tried': 24,
 'better': 25,
 'also': 26,
 'well': 27,
 'chocolate': 28,
 'make': 29,
 'try': 30,
 'eat': 31,
 'dog': 32,
 'even': 33,
 'bought': 34,
 'found': 35,
 'delicious': 36,
 'could': 37,
 'order': 38,
 'sweet': 39,
 'cup': 40,
 'drink': 41,
 'tastes': 42,
 'recommend': 43,
 'loves': 44,
 'bag': 45,
 'used': 46,
 'cannot': 47,
 'sugar': 48,
 'first': 49,
 'favorite': 50,
 'store': 51,
 'free': 52,
 'made': 53,
 'nice': 54,
 'way': 55,
 'go': 56,
 'box': 57,
 'got': 58,
 'think': 59,
 'perfect': 60,
 'mix': 61,
 'water': 62,
 'dogs': 63,
 'since': 64,
 'give': 65,
 'ordered': 66,
 'snack': 67,
 'day': 68,
 'bit': 69,
 'easy': 70,
 'two': 71,
 'many': 72,
 'ever': 73,
 'hot': 74,
 'f

In [191]:
print(srcToken.document_count)

54448


In [192]:
totalCnt=len(srcToken.word_index)

In [210]:
totalFreq=0 # 전체 단어 빈도수 총 합
rCnt=0 # 빈도수가 10 미만인 단어의 개수
rFreq=0 # 빈도수가 10 미만인 단어빈도수 총 합
for k,v in srcToken.word_counts.items():
    totalFreq+=v
    if (v<10): 
        rCnt+=1
        rFreq+=v
print("단어 빈도수 총합", totalFreq)
print("단어수",totalCnt)
print(rCnt/totalCnt)
print(rFreq/totalFreq)
print(totalCnt-rCnt) # 6956 -> 7000

단어 빈도수 총합 1323945
단어수 32668
0.7870699155136525
0.04192696826529803
6956


In [211]:
srcVocab=7000
srcToken=Tokenizer(num_words=srcVocab)
srcToken.fit_on_texts(xTrain)

In [212]:
xTrain=srcToken.texts_to_sequences(xTrain)
xTest=srcToken.texts_to_sequences(xTest)

In [216]:
print(xTrain[0])

[105, 170, 14, 491, 523, 7, 71, 1106, 45, 249, 1705, 170, 51, 33, 534, 1479, 718, 143, 168, 160, 135]


In [217]:
tarToken=Tokenizer()
tarToken.fit_on_texts(yTrain)

In [218]:
totalFreq=0 # 전체 단어 빈도수 총 합
rCnt=0 # 빈도수가 10 미만인 단어의 개수
rFreq=0 # 빈도수가 10 미만인 단어빈도수 총 합
for k,v in tarToken.word_counts.items():
    totalFreq+=v
    if (v<10): 
        rCnt+=1
        rFreq+=v
print("단어 빈도수 총합", totalFreq) # 254334
print("단어수",totalCnt) # 32668
print(rCnt/totalCnt) # 28
print(rFreq/totalFreq) # 7
print(totalCnt-rCnt) # 6956 -> 7000

단어 빈도수 총합 254334
단어수 32668
0.28223337822946004
0.07756336156392775
23448


In [219]:
tarVoc=2000
tarTokenizer=Tokenizer(num_words=tarVoc)
tarTokenizer.fit_on_texts(yTrain)

In [220]:
yTrain=tarTokenizer.texts_to_sequences(yTrain)
yTest=tarTokenizer.texts_to_sequences(yTest)

In [221]:
yTrain[:5]

[[1, 44, 5, 2],
 [1, 16, 2],
 [1, 10, 735, 8, 2],
 [1, 33, 801, 85, 1300, 2],
 [1, 187, 234, 183, 54, 31, 2]]

In [222]:
dropTrain=[i for i, sent in enumerate(yTrain) if len(sent)==2 ]
dropTest=[i for i, sent in enumerate(yTest) if len(sent)==2 ]

In [223]:
xTrain=np.delete(xTrain, dropTrain, axis=0)
yTrain=np.delete(yTrain, dropTrain, axis=0)
xTest=np.delete(xTest, dropTest, axis=0)
yTest=np.delete(yTest, dropTest, axis=0)

In [225]:
len(xTrain) # 52722
len(xTest) # 13181

13181

In [226]:
xTrain=pad_sequences(xTrain, maxlen=textMaxLen, padding='post')
xTest=pad_sequences(xTest, maxlen=textMaxLen, padding='post')
yTrain=pad_sequences(yTrain, maxlen=summaryMaxLen, padding='post')
yTest=pad_sequences(yTest, maxlen=summaryMaxLen, padding='post')

In [227]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 128
hidden_size = 256

# 인코더
encoder_inputs = Input(shape=(textMaxLen,))

# 인코더의 임베딩 층
enc_emb = Embedding(srcVocab, embedding_dim)(encoder_inputs)

# 인코더의 LSTM 1
encoder_lstm1 = LSTM(hidden_size, return_sequences=True, return_state=True ,dropout = 0.4, recurrent_dropout = 0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

# 인코더의 LSTM 2
encoder_lstm2 = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

# 인코더의 LSTM 3
encoder_lstm3 = LSTM(hidden_size, return_state=True, return_sequences=True, dropout=0.4, recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# 디코더
decoder_inputs = Input(shape=(None,))

# 디코더의 임베딩 층
dec_emb = Embedding(tarVoc, embedding_dim)(decoder_inputs)

# 디코더의 LSTM
decoder_lstm = LSTM(hidden_size, return_sequences = True, return_state = True, dropout = 0.4, recurrent_dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state = [state_h, state_c])

# 디코더의 출력층
decoder_softmax_layer = Dense(tarVoc, activation = 'softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_outputs) 

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 128)      896000      input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 50, 256), (N 394240      embedding[0][0]                  
_____________________________________________________________________________________________

In [230]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/thushv89/attention_keras/master/layers/attention.py", filename="attention.py")
from attention import AttentionLayer

# 어텐션 층(어텐션 함수)
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# 어텐션의 결과와 디코더의 hidden state들을 연결
decoder_concat_input = Concatenate(axis = -1, name='concat_layer')([decoder_outputs, attn_out])

# 디코더의 출력층
decoder_softmax_layer = Dense(tarVoc, activation='softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_concat_input)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 2)
history = model.fit([xTrain, yTrain[:,:-1]], yTrain.reshape(yTrain.shape[0], yTrain.shape[1], 1)[:,1:] \
                  ,epochs=50, callbacks=[es], batch_size = 256, validation_data=([xTest, yTest[:,:-1]], \
                  yTest.reshape(yTest.shape[0], yTest.shape[1], 1)[:,1:]))

plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 128)      896000      input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 50, 256), (N 394240      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
____________________________________________________________________________________________

KeyboardInterrupt: 

In [231]:
from keras.models import load_model

src_index_to_word = srcToken.index_word # 원문 단어 집합에서 정수 -> 단어를 얻음
tar_word_to_index = tarToken.word_index # 요약 단어 집합에서 단어 -> 정수를 얻음
tar_index_to_word = tarToken.index_word # 요약 단어 집합에서 정수 -> 단어를 얻음

# 인코더 설계
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs, state_h, state_c])

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(hidden_size,))
decoder_state_input_c = Input(shape=(hidden_size,))

dec_emb2= dec_emb_layer(decoder_inputs) 
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# 어텐션 함수
decoder_hidden_state_input = Input(shape=(textMaxLen, hidden_size))
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# 디코더의 출력층
decoder_outputs2 = decoder_softmax_layer(decoder_inf_concat) 

# 최종 디코더 모델
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)
    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tarVoce))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition: #stop_condition이 True가 될 때까지 루프 반복
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > maxTarLen):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트 합니다.
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 상태를 업데이트 합니다.
        states_value = [h, c]

    return decoded_sentence

# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2text(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + src_index_to_word[i]+' '
    return temp

# 요약문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2summary(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostoken']) and i!=target_word_index['eostoken']):
            temp = temp + tar_index_to_word[i] + ' '
    return temp

for i in range(500, 1000):
    print("원문 : ",seq2text(xTest[i]))
    print("실제 요약문 :",seq2summary(yTest[i]))
    print("예측 요약문 :",decode_sequence(xTest[i].reshape(1, textMaxLen)))
    print("\n")

TypeError: __init__() got an unexpected keyword argument 'ragged'

In [229]:
model.save('review_seq2seq_Multi_LSTM.h5')


In [233]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

model = load_model('review_seq2seq_Multi_LSTM.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


ValueError: Unknown layer: AttentionLayer